In [1]:
import xarray as xr
import geopandas as gpd
import rasterio.features
import pandas as pd

In [3]:
# read precipitation information
precip_xr = xr.open_dataset(r'C:\Users\hadunka2\Box\precip_district_for_protensia\precip\chirps-v2.0.monthly.nc')['precip'].\
                        sel(latitude = slice(-19, -6), longitude = slice(21, 35), time = slice("2000", "2023"))                        

In [4]:
precip_xr

<xarray.DataArray 'precip' (time: 280, latitude: 260, longitude: 280)>
[20384000 values with dtype=float32]
Coordinates:
  * latitude   (latitude) float32 -18.98 -18.93 -18.88 ... -6.125 -6.075 -6.025
  * longitude  (longitude) float32 21.02 21.07 21.12 21.18 ... 34.88 34.93 34.97
  * time       (time) datetime64[ns] 2000-01-01 2000-02-01 ... 2023-04-01
Attributes:
    units:               mm/month
    standard_name:       convective precipitation rate
    long_name:           Climate Hazards group InfraRed Precipitation with St...
    time_step:           month
    geostatial_lat_min:  -50.0
    geostatial_lat_max:  50.0
    geostatial_lon_min:  -180.0
    geostatial_lon_max:  180.0

In [6]:
# Load the GeoDataFrame with polygons of interest
zambia_2010_boundary = gpd.read_file(r'C:\Users\hadunka2\Box\precip_district_for_protensia\precip\district_zambia_2010\vn685gp2885.shp')

In [7]:
precip_data = []
for district in range(len(zambia_2010_boundary)):
    print(district)
    for year in range(2000, 2023):
        for month in ['01', '02', '03','04', '05', '06', '07', '08', '09','10', '11', '12']:
            gdf = zambia_2010_boundary.iloc[[district]]
            # Create a mask for the polygons
            precip_xr_tmp = precip_xr.sel(time = str(year)+'-'+month+'-01')
            mask = rasterio.features.geometry_mask(gdf.geometry,\
                        out_shape=precip_xr_tmp.shape,
                        transform=rasterio.transform.from_bounds(west = 21.0249939, south = -6.02500153 , east = 34.97499084 , north =-18.97500038 ,
                                            width=len(precip_xr_tmp.longitude.values),
                                            height=len(precip_xr_tmp.latitude.values)), 
                                                   invert=True)
            mask = xr.DataArray(mask, dims=("latitude", "longitude"))
            # Apply the mask to the precipitation data
            precipitation_masked_polygons = precip_xr_tmp.where(mask)
            precipitation_mean = precipitation_masked_polygons.mean().values
            precip_data.append({
            'District': gdf.laa.values[0],
            'year': year,
            'month': int(month),
            'precip': precipitation_mean
        })
            

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73


In [9]:
pd.DataFrame(precip_data).to_csv(r'precip_month_district.csv',index = False)